# National Data Science Bowl - Plankton

## Action Plan

* Make overfitting model
* Data augmentation
* Batch normalization
* Dropout
* Ensembling

## Imports and Directories

In [1]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
SCRIPTS_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data'

In [2]:
#import modules
from utils import *
%matplotlib inline

Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 85.0% of memory, cuDNN 5105)
/home/nathan/anaconda3/envs/deepLearning/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/nathan/git/planktonDataScienceBowl/scripts/data


## Simple Model

#### Constants

In [4]:
img_rows, img_cols = 128, 128
in_shape = (img_rows, img_cols)
batch_size = 32
nb_classes = 121

#### Batches

In [5]:
gen = image.ImageDataGenerator(rescale=1. / 255)
# gen = image.ImageDataGenerator()

In [6]:
train_batches = get_batches(train_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)
val_batches   = get_batches(valid_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)

Found 27184 images belonging to 121 classes.
Found 3152 images belonging to 121 classes.


#### Model

In [7]:
def get_model():
    model = Sequential([
        Convolution2D(64,3,3, border_mode='same', activation='relu', input_shape=(1, img_rows, img_cols)),
        Convolution2D(64,3,3, border_mode='same', activation='relu'),
        MaxPooling2D(),
        Convolution2D(128,3,3, border_mode='same', activation='relu'),
        MaxPooling2D(),
        Convolution2D(256,3,3, border_mode='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(2048, activation='relu'),
        Dense(2048, activation='relu'),
        Dense(nb_classes, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model = get_model()

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 128, 128)  640         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 128, 128)  36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 64, 64)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 128, 64, 64)   73856       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

#### Train

In [10]:
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=1,
                    validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample,
                    verbose=True)

Epoch 1/1
27184/27184 [==============================] - 139s - loss: 2.3076 - acc: 0.4036 - val_loss: 1.6878 - val_acc: 0.5260


In [11]:
model.optimizer.lr=0.1
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=1,
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample,
                    verbose=True)

Epoch 1/1
27184/27184 [==============================] - 139s - loss: 1.3828 - acc: 0.5935 - val_loss: 1.4815 - val_acc: 0.5644


In [12]:
model.optimizer.lr=0.01
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=4,
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample,
                    verbose=True)

Epoch 1/4
27184/27184 [==============================] - 139s - loss: 0.9279 - acc: 0.7066 - val_loss: 1.5587 - val_acc: 0.5971
Epoch 2/4
27184/27184 [==============================] - 139s - loss: 0.5387 - acc: 0.8185 - val_loss: 1.7641 - val_acc: 0.5850
Epoch 3/4
27184/27184 [==============================] - 139s - loss: 0.2956 - acc: 0.8992 - val_loss: 2.2239 - val_acc: 0.5863
Epoch 4/4
27184/27184 [==============================] - 139s - loss: 0.1840 - acc: 0.9416 - val_loss: 2.4738 - val_acc: 0.5825


## Initial overfitting model

### Constants

In [4]:
img_rows, img_cols = 128, 128
in_shape = (img_rows, img_cols)
batch_size = 32
nb_classes = 121

### Batches

In [5]:
gen = image.ImageDataGenerator(rescale=1. / 255)

In [6]:
train_batches = get_batches(train_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)
val_batches   = get_batches(valid_path, batch_size=batch_size, 
                            target_size=in_shape, color_mode="grayscale", 
                            gen=gen)

Found 27184 images belonging to 121 classes.
Found 3152 images belonging to 121 classes.


### Model

In [7]:
model = Sequential()

#### Input layer

In [8]:
model.add(Convolution2D(64, 3, 3, border_mode='same', input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))

#### Convolution layers

In [9]:
def convBlock(model, layers, filters):
    for i in range(layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))

In [10]:
convBlock(model, 1, 64)
convBlock(model, 2, 128)
convBlock(model, 3, 256)
convBlock(model, 3, 512)

In [11]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))

In [12]:
model.add(Flatten())
FCBlock(model)
FCBlock(model)
model.add(Dense(nb_classes, activation='softmax'))

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [14]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 128, 128)  640         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 128, 128)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 128, 128)  36928       activation_1[0][0]               
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 64, 64)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

### Train Model

In [15]:
model.fit_generator(train_batches, samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=1,
                    validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample,
                    verbose=True)

Epoch 1/1
27184/27184 [==============================] - 313s - loss: 4.1907 - acc: 0.0651 - val_loss: 4.2187 - val_acc: 0.0615


In [ ]:
model.optimizer.lr=0.1
model.fit_generator(train_batches, samples_per_epoch=27184,
                    nb_epoch=1,
                    validation_data=val_batches, nb_val_samples=3152,
                    verbose=True)

In [ ]:
model.optimizer.lr=0.01
model.fit_generator(train_batches, samples_per_epoch=27184,
                    nb_epoch=100,
                    validation_data=val_batches, nb_val_samples=3152,
                    verbose=True)

## Data Augmentation

In [ ]:
# image_generator = image.ImageDataGenerator(
#                 rotation_range=360,
#                 width_shift_range=0.02,
#                 height_shift_range=0.02,
#                 shear_range=0.05,
#                 zoom_range=0.05)

## Batch Normalization 

## Dropout 

## Ensembling 